In [2]:
import autogen

In [ ]:
config_list = [
    {
    }
]

llm_config = {
    "timeout": 600,
    "cache_seed": 42,  # Use 'cache_seed' instead of 'use_cache' (True becomes a seed value)
    "config_list": config_list,
}

In [3]:
tasks = [
    """On which days in 2024 was Microsoft Stock higher than $400? Comment on the stock performance.""",
    """Make a pleasant joke about it.""",
]

In [4]:
inner_assitant = autogen.AssistantAgent(
    "Inner Assitant",
    llm_config = llm_config,
    is_termination_msg = lambda x : x.get("content","").find("TERMINATE") >=0
)

In [6]:
inner_code_interpretor = autogen.UserProxyAgent(
    "Inner-code interpretor",
    llm_config = llm_config,
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir":"coding",
        "use_docker":False
    },
    default_auto_reply = "",
    is_termination_msg = lambda x: x.get("content", "").rstrip().find("TERMINATE") >= 0,

)

In [7]:
group_chat = autogen.GroupChat(
    agents = [inner_assitant,inner_code_interpretor],
    messages = [],
    max_round = 10,
    speaker_selection_method = "round_robin",
    allow_repeat_speaker = False,
    
)

In [9]:
manager = autogen.GroupChatManager(
    groupchat=group_chat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
)

# Inner and outer level individual agents

In [11]:
assitant_1 = autogen.AssistantAgent(
    name="Assitant_1",
    llm_config=llm_config,
    
)

In [12]:
assistant_2 = autogen.AssistantAgent(
    name = "Assitant_2",
    llm_config = llm_config
)

In [13]:
writer = autogen.AssistantAgent(
    name = "Writer",
    llm_config = llm_config,
    system_message = "You are a professional writer, known for your insightful and engaging articles. You transform complex concepts into clear and concise articles."
)

In [14]:
reviewer = autogen.AssistantAgent(
    name="Reviewer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a compliance reviewer, known for your thoroughness and commitment to standards.
    Your task is to scrutinize content for any harmful elements or regulatory violations, ensuring
    all materials align with required guidelines.
    You must review carefully, identify potential issues, and maintain the integrity of the organization.
    Your role demands fairness, a deep understanding of regulations, and a focus on protecting against
    harm while upholding a culture of responsibility.
    """,
)


In [15]:
user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

In [16]:
def writing_message(recipient, messages, sender, config):
    return f"Polish the content to make an engaging and nicely formatted blog post. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"


In [17]:
nested_chat_queue = [
    {"recipient": manager, "summary_method": "reflection_with_llm"},
    {"recipient": writer, "message": writing_message, "summary_method": "last_msg", "max_turns": 1},
    {"recipient": reviewer, "message": "Review the content provided.", "summary_method": "last_msg", "max_turns": 1},
    {"recipient": writer, "message": writing_message, "summary_method": "last_msg", "max_turns": 1},
]

In [18]:
assitant_1.register_nested_chats(
    nested_chat_queue,
    trigger = user
)

In [ ]:
res = user.initiate_chats(
    [
        {"recipient": assitant_1, "message": tasks[0], "max_turns": 1, "summary_method": "last_msg"},
        {"recipient": assistant_2, "message": tasks[1]},
    ]
)